In [1]:
api_key = ''

In [2]:
from datasets import load_from_disk

data = load_from_disk('dataset-fixed')

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [16]:
n = 2
prompt = 'Döntsd el a következő cikkről, hogy korrupcióhoz kapcsolódó vagy egyéb témáról szól!\n\n'
for i in [1, 2, 3, 5]:
    prompt += ''+data['train'][i]['title'] + '\n' +'téma: '+('egyéb' if data['train'][i]['theme'] == 0 else 'korrupció') + '\n\n'
    #+'\n\ncikk: ' + data['train'][2]['title'] + '\n' +'###\ntéma: korrupció'+'\n\ncikk: ' + data['train'][4]['title'] + '\n' +'###\ntéma: korrupció'+'\n\ncikk: ' + data['train'][6]['title'] + '\n' +'###\ntéma: egyéb'+'\n\ncikk: ' + data['test'][n]['title'] + '\n' +'###\ntéma:'
prompt += '' + data['test'][n]['title'] + '\n' +'téma:'
print(prompt)

Döntsd el a következő cikkekről, hogy korrupcióhoz kapcsolódnak vagy egyéb témáról szólnak!

Kinéztek a repülő ablakán és egy észak-koreai rakétát láttak
téma: egyéb

Belföld: Szerelmi szál a Zuschlag-ügyben
téma: korrupció

A belső elhárítás tudott a békési rendőrügyekről
téma: korrupció

Megváltoztatja a NER megjelenése a magyar Mekit
téma: egyéb

Belföld: Novemberre idézi be a bíró Gyurcsány Ferencet
téma:


In [19]:
def trunc_to(descr, n):
    if '.' in descr[int(n/2):n]:
        return descr[:descr[:n].rfind('.')+1]
    elif ' ' in descr[int(n/2):n]:
        return descr[:descr[:n].rfind(' ')+1]
    else:
        return descr[:n]


In [47]:
import os
import openai
from time import sleep
from tqdm import tqdm

f = open('resp-50.txt', 'w')
responses = list()
labels = list()
openai.api_key = api_key
for n in tqdm(range(100)):
    prompt = 'Döntsd el a következő cikkekről, hogy korrupcióhoz kapcsolódnak vagy egyéb témáról szólnak!\n\n'
    for i in [1, 2, 3, 4]:
        prompt += ''+data['train'][i]['title'] + '\n' +'téma: '+('egyéb' if data['train'][i]['theme'] == 0 else 'korrupció') + '\n\n'
        #+'\n\ncikk: ' + data['train'][2]['title'] + '\n' +'###\ntéma: korrupció'+'\n\ncikk: ' + data['train'][4]['title'] + '\n' +'###\ntéma: korrupció'+'\n\ncikk: ' + data['train'][6]['title'] + '\n' +'###\ntéma: egyéb'+'\n\ncikk: ' + data['test'][n]['title'] + '\n' +'###\ntéma:'
    prompt += '' + data['test'][n]['title'] + '\n' +'téma:'
    response = None
    while True:
        try:
            response = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt,
              temperature=0,
              max_tokens=6,
              top_p=1,
              frequency_penalty=0.5,
              presence_penalty=0
            )
            sleep(1)
            break
        except Exception:
            print('error')
            sleep(1)
    resp = response["choices"][0]["text"]
    responses.append(resp)
    labels.append(data['test']['theme'][n])
    f.write(str(data['test']['theme'][n])+' '+resp+'\n')
f.close()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:09<00:00,  1.89s/it]


In [48]:
list(zip(responses, labels))

[(' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 1),
 (' egyéb', 1),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' korrupció', 1),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' korrupció', 1),
 (' korrupció', 0),
 (' korrupció', 1),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' korrupció', 0),
 (' egyéb', 1),
 (' korrupció', 1),
 (' egyéb', 0),
 (' egyéb', 0),
 (' egyéb', 1),
 (' egyéb', 0),
 (' korrupció', 0),
 (' egyéb', 0),
 (' korrupció', 1),
 (' korrupció', 

In [49]:
n = 0
c = 0
for r, l in zip(responses, labels):
    n += 1
    if (l == 1 and 'korrupció' in r) or (l == 0 and 'korrupció' not in r):
        c += 1
print(n, c)

100 87


In [43]:
n = 0
c = 0
for r, l in zip(responses, labels):
    n += 1
    if (l == 1 and 'igen' in r.lower()) or (l == 0 and 'igen' not in r.lower()):
        c += 1
print(n, c)

50 33
